# conda environment note - unrelated to forecasting

ISSUE: package (pycaret) wasnt in the channels i had, so this helped. Channels are basically servers people host there packages on

https://stackoverflow.com/questions/48493505/packagesnotfounderror-the-following-packages-are-not-available-from-current-cha

'conda config --append channels conda-forge'

This tells conda to also look on the conda-forge channel when you search for packages... this is usually a good place to start

* use the Anaconda Prompt terminal to perform all anaconda needs:

'conda info' - info about conda including python version
'conda list' - list all packages
'conda list pycaret$' - list certain package
'conda install pycaret' - install a package
'conda update pycaret' - update package
'conda remove pyarrow' - remove package
'conda update --all' - update all packagesy

# VM Forecasting
In Python

In [27]:
# Packages
import snowflake.connector
import pandas as pd
import os
import numpy as np

from pycaret.regression import * # import the regression module
import plotly.express as px

In [ ]:
# copy and paste in to a new chunk, enter credentials and run to save in environment. Then delete chunk
%env snowflakeuser=<your_snowflake_username>
%env snowflakepass=<your_snowflake_password>

In [30]:
# Query Snowflake
def snowflake_to_pandas(connection_params, query):
    try:
        # Establish a connection to Snowflake
        conn = snowflake.connector.connect(**connection_params)

        # Execute the SQL query and fetch the results into a DataFrame
        df = pd.read_sql_query(query, conn)

        # Close the connection
        conn.close()

        return df
    except Exception as e:
        print(f"Error: {e}")
        return None

# Snowflake connection parameters
connection_params = {
    "user": os.environ['snowflakeuser'],
    "password": os.environ['snowflakepass'],
    "account": "zib52348.us-east-1",
    "role": "ACCOUNTADMIN",
    "warehouse": "REPORTING",
    "database": "ANALYTICS",
    "schema": "FORECASTING",
}

# SQL command 
query = 'SELECT * FROM "ANALYTICS"."FORECASTING"."sales_fcast_combined_v"'

# Call the function to retrieve the data into a Pandas DataFrame
result_df = snowflake_to_pandas(connection_params, query)

if result_df is not None:
    print(result_df.head())  # Display the first few rows of the DataFrame
else:
    print("Failed to retrieve data from Snowflake.")

   DEP_ENT       MONTH  NET_SALES  BUDGET_AMOUNT      FORECAST
0  240_155  2023-09-01        NaN      5373043.0  5.613731e+06
1  240_155  2023-10-01        NaN      5376636.0  4.859158e+06
2  240_155  2023-11-01        NaN      5295121.0  4.584387e+06
3  240_155  2023-12-01        NaN      5313648.0  4.532230e+06
4  250_155  2016-01-01  3738317.4      6415527.0           NaN


In [31]:
# head and tail
print(result_df)

      DEP_ENT       MONTH   NET_SALES  BUDGET_AMOUNT      FORECAST
0     240_155  2023-09-01         NaN      5373043.0  5.613731e+06
1     240_155  2023-10-01         NaN      5376636.0  4.859158e+06
2     240_155  2023-11-01         NaN      5295121.0  4.584387e+06
3     240_155  2023-12-01         NaN      5313648.0  4.532230e+06
4     250_155  2016-01-01  3738317.40      6415527.0           NaN
...       ...         ...         ...            ...           ...
1260  210_155  2023-02-01  6204353.87      4631525.0  8.047390e+06
1261  210_155  2023-03-01  5973971.86      6887883.0  8.047390e+06
1262  210_155  2023-04-01  5560797.29      7108739.0  8.047390e+06
1263  210_155  2023-05-01  4008775.43      5308974.0  6.709869e+06
1264  210_155  2023-06-01  2741890.24      5559823.0  8.037976e+06

[1265 rows x 5 columns]


In [33]:
df_200_155 = result_df[result_df["DEP_ENT"] == '200_155']
df_200_155 = df_200_155[df_200_155['MONTH'] <= pd.Timestamp(2023,6,1)] 
"""
ABOVE: we have data beyond july '23, but we want to compare with 
some models that only had through june. No way to check model accuracy beyond the daya we have
"""
df_200_155 = df_200_155[["MONTH", "NET_SALES"]]
print(df_200_155)
print(df_200_155.dtypes)

         MONTH   NET_SALES
158 2019-10-01  1522283.25
159 2019-11-01  2681924.32
160 2019-12-01  4944662.33
161 2020-01-01  1729231.23
162 2020-02-01  1557584.48
..         ...         ...
628 2019-05-01  2333062.18
629 2019-06-01  1856416.87
630 2019-07-01  1895360.33
631 2019-08-01  1955738.19
632 2019-09-01  1749430.60

[90 rows x 2 columns]
MONTH        datetime64[ns]
NET_SALES           float64
dtype: object


In [ ]:
result_df.set_index('MONTH', drop = True, inplace)

# Time Series with pycaret
https://moez-62905.medium.com/pycaret-3-is-coming-whats-new-8d7d241c40d8